In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Метрика качества:

In [2]:
def calculate_metrics(self, submit):
    df = self.answers.join(submit, on='id')

    y_true = []
    y_scores = []
    for idx, row in df.iterrows():
        answer_labels = row['labels'].split(',')
        y_true.append([1 if label in answer_labels else 0 for label in LABELS])
        y_scores.append([row['normal'], row['insult'], row['obscenity'], row['threat']])

## Чтение данных из файла txt

In [2]:
df = pd.DataFrame(columns = [['id', 'text', 'normal','insult','obscenity','threat']]);
j=0;

f = open("data/train.txt", "r", encoding='utf-8')
for line in f:
    nor, ins, obs, thr = [0,0,0,0]
    l = line.split('\t')
    em_List = (l[1:-1])
    for i in em_List:
        if i=='__label__NORMAL':
            nor+=1
        elif i=='__label__INSULT':
            ins+=1
        elif i=='__label__OBSCENITY':
            obs+=1
        else:
            thr+=1
    df.loc[j]=[int(l[0]),l[-1][:-1],nor,ins,obs,thr]
    j += 1
f.close()

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
train_file_lenght=148775

In [ ]:
df.head(2000).to_csv('short_train.csv')

Файл очень большой, поэтомц работать пока буду только с первыми 2000 элементами, которые вынесены в отдельный файд short_train.csv

In [3]:
df = pd.read_csv('short_train.csv', usecols=['id', 'text', 'normal','insult','obscenity','threat'])
df.tail()

id                                               text  normal  \
1995   15834  людей искиства тянет ,он не спать не есть небу...       1   
1996   64370  какой мудак старое кино переименновывает посто...       0   
1997   17080           спит он под кондиционером! я в восторге!       1   
1998  237178    могу пнуть по яйцам и отлетит дебил твоя голова       0   
1999   26969  юра ефремов подонок и негодяй к тому же подлей...       0   

      insult  obscenity  threat  
1995       0          0       0  
1996       1          0       0  
1997       0          0       0  
1998       1          0       1  
1999       1          0       0

# Обработка текста

S.lower

In [8]:
import re
from pymorphy2 import MorphAnalyzer

In [6]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

In [9]:
morph = MorphAnalyzer()

In [10]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        token = token.strip()
        token = morph.normal_forms(token)[0]
            
        tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [12]:
lemmatize(df.iloc[0]['text'])

['дворник', 'надо', 'тоже', 'уничтожить']

In [14]:
data = df['text'].apply(lemmatize)

In [15]:
data.head()

0                    [дворник, надо, тоже, уничтожить]
1    [мой, старший, неделя, шипеть, не, принимать, ...
2                        [полностью, с, вы, согласный]
3          [хоть, нога, вверх, ничего, не, измениться]
4                     [а, что, значит, левый, ребёнок]
Name: text, dtype: object

## Разделение на обучающую и тестовую выборки

In [140]:
from sklearn.model_selection import train_test_split

In [141]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], 
                                                    df.drop('text', axis=1), 
                                                    test_size=0.33,
                                                    random_state=42)

In [143]:
y_train

id  normal  insult  obscenity  threat
81    180341       0       1          0       0
915   241550       0       1          0       0
1018   45802       0       1          0       0
380   178744       0       0          0       1
1029   76653       1       0          0       0
...      ...     ...     ...        ...     ...
1130  240196       1       0          0       0
1294   80655       1       0          0       0
860    82203       1       0          0       0
1459  193207       1       0          0       0
1126  218468       1       0          0       0

[1340 rows x 5 columns]

In [142]:
X_train

81                    рот свой закрой подстилка путинская
915     рады? то есть они раньше, до пандемии, вынужде...
1018    как , они заебли сука когда же нажрутся падлы ...
380                                     сдохни по скорее.
1029    здравствуйте, вам срочно кизил нужен? у меня б...
                              ...                        
1130                                 офигенная бобомчка 😲
1294     кошмар. ты срань интернетовская ,посмотри на ...
860     а что делать,если заяц прописался на дачах?выг...
1459                         и серпатые, и торцовые-класс
1126    познакомилась с парнем.он сказал...пока платье...
Name: text, Length: 1340, dtype: object

## Преобразование текстовых данных   
#### CountVectorizer

In [139]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [149]:
comments_vec_train = count_vec.fit_transform(X_train)

In [150]:
comments_vec_train

<1340x6575 sparse matrix of type '<class 'numpy.int64'>'
	with 13895 stored elements in Compressed Sparse Row format>

In [151]:
comments_vec_test = count_vec.transform(X_test)

In [152]:
from sklearn.linear_model import LogisticRegression

In [155]:
model1 = LogisticRegression()
model1.fit(comments_vec_train,y_train['normal'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [160]:
model2 = LogisticRegression()
model2.fit(comments_vec_train,y_train['insult'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
model3 = LogisticRegression()
model3.fit(comments_vec_train,y_train['obscenity'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [162]:
model4 = LogisticRegression()
model4.fit(comments_vec_train,y_train['threat'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [163]:
p1 = model1.predict(comments_vec_test)
p2 = model2.predict(comments_vec_test)
p3 = model3.predict(comments_vec_test)
p4 = model4.predict(comments_vec_test)

In [166]:
from sklearn.metrics import f1_score

In [170]:
f1_score(y_test['normal'], p1)

0.921875

In [171]:
F1 = np.array([f1_score(y_test['normal'], p1),
         f1_score(y_test['insult'], p2),
         f1_score(y_test['obscenity'], p3),
         f1_score(y_test['threat'], p4),

])
F1.mean() # F1

0.341399485930736